# Comparison of Standard IF to Cyclic IF

**Question:** How does the signal-to-background ratio and staining specificity compare in standard IF vs. cyclic IF?

**Samples:** 
- Tissue ID 44290: HER2+/ER+ breast cancer. Section 112 stained with a 5 round cyclic IF protocol, sections 113 to 116 stained with the same antibodies in a standard IF protocol.
- Tissue ID 44294: Adjacent normal breast from the patient above. Section 116 is cyclic, section 117 to 120 are standard IF.

**Method**: For each stain, pixel intensity was manually thresholded to separate the positive pixels from the negative. Visualizations produced include:
1. Autoscaled overview of stain, with dynamic range of 0 - 99th percentile displayed in legend.
2. Mask of positive pixels, so that masks from standard and corresponding cyclic stains can be compared for specificity.
3. Measure mean intensity in mask area and background, and output values to .csv for downstream analysis.

### reviewer comments

Define in legend the scale (e.g. scale bar needed) for the tissue images shown throughout 

In [ ]:
pwd

In [ ]:
#paths
import os
#os.chdir('/home/groups/graylab_share/OMERO.rdsStore/engje/Data/cycIF_ValidationStudies/cycIF_Validation')
codedir = os.getcwd()

In [ ]:
#libraries 
import importlib
import util
import pandas as pd
import numpy as np
import shutil
import skimage
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import patches
#os.chdir('/home/groups/graylab_share/OMERO.rdsStore/engje/Data/mplex_image')
#import analyze, mpimage #, cmif

In [ ]:
os.chdir(codedir)
%matplotlib inline
rootdir = '/home/groups/graylab_share/OMERO.rdsStore/engje/Data/cycIF_ValidationStudies/cycIF_Validation'

## Accessing Images

To run this code, first download necessary images from synapse.org, ID =  'syn23644791'

https://www.synapse.org/#!Synapse:syn23644791


In [ ]:
importlib.reload(util)

In [ ]:
#analyze regions in crop (20191209)
%matplotlib inline
d_crop = {'44290-112':(5000,8800,1300,1800),
 '44290-113':(5886,8111,1300,1800),
 '44290-114':(5010,9242,1300,1800),
 '44290-115':(5975,10490,1300,1800),
 '44290-116':(6336,9174,1300,1800),
 '44294-116':(9547,5459,1300,1800),
 '44294-117':(12180,6092, 1300,1800),
 '44294-118':(10901, 6582, 1300,1800),
 '44294-119':(9853,5557, 1300,1800),
 '44294-120':(10250,5510, 1300,1800),
 }

d_rename = {#'Her2':'HER2', 'pH3':'pHH3', 'LaminAC':'LamAC',
 '(R2)':'R2', '(R3)':'R3', '(R4)':'R4', '(R5)':'R5', '(R6)':'R6'}

s_thresh='minimum90' #'minimum'#
k=4
df_result = pd.DataFrame()
dd_result = {}
#run
d_process = {'44290-112':f'{rootdir}/Images/tiff/44290/44290-112',
    '44290':f'{rootdir}/Images/tiff/44290',
    '44290-116':f'{rootdir}/Images/tiff/44290/44294-116',
 }

for idx,(s_sample, s_path) in enumerate(d_process.items()):
    print(s_sample)
    os.chdir(s_path)
    df_img = util.parse_org()
    df_img = df_img.replace(d_rename)
    df_img['slide_marker'] = df_img.scene + '_' + df_img.marker
    df_img['img_index'] = df_img.index
    df_thresh = pd.read_csv(f'{rootdir}/Metadata/44290/metadata_single_vs_cyclic.csv',index_col=0)
    df_thresh['slide_marker'] = df_thresh.slide + '_' + df_thresh.marker
    df_thresh = df_img.merge(df_thresh, how='inner', on='slide_marker',suffixes=('','_y'))
    df_thresh.index = df_thresh.img_index
    df_thresh['minimum110'] = df_thresh.minimum + df_thresh.minimum/10
    df_thresh['minimum90'] = df_thresh.minimum - df_thresh.minimum/10
    #results
    for s_marker in sorted(set(df_thresh.marker)):      
        df_marker = df_thresh[(df_thresh.marker==s_marker) & (df_thresh.rounds !='R6')]
        print(f'{s_marker}  {len(df_marker)}')
        #plot the images
        #fig = util.array_img(df_marker.sort_values('condition'),s_xlabel='marker',ls_ylabel=['scene','color'],s_title='condition',tu_array=(2,len(df_marker)//2),tu_fig=(8,8),cmap='inferno')
        #fig.savefig(f'{codedir}/Figures/{s_sample}/SinglevsCyclic_TissueLoss_Background_{s_marker}.png')
        df_marker_thresh,d_mask = util.thresh_erode(df_marker.sort_values('condition'),d_crop,s_thresh=s_thresh,k=k)
        df_result=df_result.append(df_marker_thresh)
        dd_result.update({f'{s_sample}_{s_marker}':d_mask})
        #plot the mask
        if s_marker == 'CD44':
            i_plot=2
            fig, ax = plt.subplots(2,i_plot,figsize=(6,8)) #len(df_marker)//2
            ax=ax.ravel()
            for idx,(s_index, a_mask) in enumerate(d_mask.items()):
                stacked_img = np.stack((a_mask[0],)*3, axis=-1).astype('uint8')
                for i in np.unique(a_mask[0]):
                    stacked_img = np.where(stacked_img == i, (i+1)*(255/(len(np.unique(a_mask[0])+1))), stacked_img)
                stacked_img = stacked_img.astype('uint8')
                stacked_img[:,:,0][1600:1633,1000:1153] = 255
                ax[idx].imshow(stacked_img)
                ax[idx].set_title(f"{df_thresh.loc[s_index,'condition']}\n{df_thresh.loc[s_index,'marker']}")
            #fig.savefig(f'{rootdir}/Figures/SinglevsCyclic_Thresholding_{s_marker}.png')
            #plot the mask (bg)
            for idx,(s_index, a_mask) in enumerate(d_mask.items()):
                stacked_img = np.stack((a_mask[1],)*3, axis=-1).astype('uint8')
                for i in np.unique(a_mask[0]):
                    stacked_img = np.where(stacked_img == i, (i+1)*(255/(len(np.unique(a_mask[0])+1))), stacked_img)
                stacked_img = stacked_img.astype('uint8')
                stacked_img[:,:,0][1600:1633,1000:1153] = 255
                ax[idx+2].imshow(stacked_img)
                ax[idx+2].set_title(f"{df_thresh.loc[s_index,'condition']}\n{df_thresh.loc[s_index,'marker']}")
            fig.savefig(f'{codedir}/Figures/SinglevsCyclic_Thresholding_{s_marker}_{s_thresh}_{k}.png')
        #break

In [ ]:
#save superpixel results
df_result.reset_index().to_csv(f'{codedir}/single_vs_cyclic_SBR_SP{k}_{s_thresh}.csv')